In [0]:
# ============================================
# 03_gold_manufacturing_defects
# - SILVER -> GOLD
# - KPIs agregados (minimalista)
# ============================================

from pyspark.sql import functions as F

#WIDGETS

In [0]:
dbutils.widgets.removeAll()

dbutils.widgets.text("catalog", "catalog_project")
dbutils.widgets.text("silver_schema", "silver")
dbutils.widgets.text("gold_schema", "gold")


In [0]:
catalog    = dbutils.widgets.get("catalog").strip()
silver_schema = dbutils.widgets.get("silver_schema").strip()
gold_schema   = dbutils.widgets.get("gold_schema").strip()


#CONTEXT

In [0]:
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_schema}")
source_table = f"{catalog}.{silver_schema}.silver_manufacturing_defects"
target_table = f"{catalog}.{gold_schema}.gold_manufacturing_defects"

#LOAD SILVER

In [0]:
df = spark.table(source_table)


#AGREGACIONES GOLD

In [0]:
df_gold = (
    df.agg(
        # Producción
        F.sum("ProductionVolume").alias("total_production_volume"),

        # Defectos
        F.sum("expected_defects").alias("total_expected_defects"),
        F.avg("DefectRate").alias("avg_defect_rate"),

        # Costos / energía
        F.avg("cost_per_unit").alias("avg_cost_per_unit"),
        F.avg("energy_per_unit").alias("avg_energy_per_unit"),

        # Seguridad
        F.avg("incident_rate_per_1k_units").alias("avg_incident_rate_per_1k_units")
    )
)


#SAVE GOLD

In [0]:
(df_gold.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .format("delta")
 .saveAsTable(target_table)
)

target_path = "abfss://gold@storageaccountcf9603.dfs.core.windows.net/gold_defects1"

(df_gold.write
  .format("delta")
  .mode("overwrite")   # o "append"
  .save(target_path)
)

#VALIDACION

In [0]:
print(f"OK: {target_table}")
display(spark.table(target_table).limit(10))